In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-indonesian-v2/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

10328

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [6]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [9]:
# tokenizer.decode(d[0]['predict_en'], decode_with_timestamps = True)

In [10]:
len(files)

10328

In [11]:
from huggingface_hub import hf_hub_download

In [12]:
f = hf_hub_download(
    repo_id="mesolitica/pseudolabel-indonesian-large-v3-timestamp", 
    filename="translated-indonesian-en.json",
    repo_type = 'dataset'
)

In [13]:
with open(f) as fopen:
    translation = json.load(fopen)

len(translation)

326191

In [14]:
keys = list(translation.keys())

In [15]:
translation[keys[0]]

"criminals for society How to not be affected by this turn I swear <> you talk a day after talking about what is difficult is still difficult, those who do not get justice are still not <> get it so rest, better focus on your life, your family, your environment, don't be"

In [16]:
keys[0]

' penjahat-penjahat untuk masyarakat Bagaimana tidak terpengaruhi dengan giliran tadi sumpah <> lu sehari habis nyobrol yang susah tetap susah yang yang nggak dapet keadilan tetap masih nggak <> dapet jadi santai mending fokus kehidup lu keluarga lu lingkungan lu jangan jangan jadi'

In [17]:
popped = []
for k in list(keys):
    v = translation[k]
    v = re.sub(r'\s-', '-', v)
    v = v.replace('< >', '<>').replace(' >', '<>')
        
    translation[k] = v
    k_split = k.split('<>')
    v_split = v.split('<>')
    
    if len(k_split) != len(v_split):
        translation.pop(k, None)
        popped.append((k_split, v_split, k, v))

In [18]:
len(popped)

39590

In [19]:
popped[0]

([' Hari raya dapat juga tunjangan ',
  ' Kita melihat loh ya ',
  ' Cewek-cewek begitu tuh ',
  ' Mereka tuh bener-bener menjaga penampilan ',
  ' Terus all out ',
  ' Dalam melayani tamu ',
  ' Karena kalau gak all out terdepak ',
  ' Istilahnya mereka tuh ',
  ' Melakukan sepenuh hati ',
  ' Atas ',
  ' Passionnya mereka ',
  ' Atau memang mereka ',
  ' Sesuatu yang ',
  ' Yaudah untuk uang aja ',
  ' Gimana tuh'],
 ["Even if it's a holiday, you can get an allowance. Let's see a girl like that. Take care of her appearance. Keep going all the way. In treating guests, they will be kicked out if they don't finish it. In other words, they do it with all their heart. Because of the passion. Or are they something that is good for money. So what?"],
 ' Hari raya dapat juga tunjangan <> Kita melihat loh ya <> Cewek-cewek begitu tuh <> Mereka tuh bener-bener menjaga penampilan <> Terus all out <> Dalam melayani tamu <> Karena kalau gak all out terdepak <> Istilahnya mereka tuh <> Melakukan s

In [20]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
                
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
                    
            
            if data[i]['score_id'] > minimum_score:
                
                a = tokenizer._decode_asr(
                [{'tokens': np.array([data[i]['predict_id']])}], 
                return_timestamps = True, return_language = 'id', 
                time_precision = 0.02)[1]['chunks']
            
                a = [a_['text'] for a_ in a]
                
                t = ' <>'.join(a)
                if t not in translation:
                    continue
                    
                t_translated = translation[t]
                t_translated_splitted = t_translated.split('<>')
                if len(t_translated_splitted) != len(a):
                    continue
                
                mapping = {}
                for k in range(len(a)):
                    s = t_translated_splitted[k].strip()
                    if a[k][0] == ' ':
                        s = ' ' + s
                    if a[k][-1] == ' ':
                        s = s + ' '
                    mapping[a[k]] = s
            
                a = np.array(data[i]['predict_id'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        if t_ in mapping:
                            t_ = mapping[t_]
                        else:
                            t_ = ''
                    
                    splitted = t_.split()
                    if len(splitted):
                        ratio = (len(set(splitted)) / len(splitted))
                        if len(t_) > 100 and ratio < 0.5:
                            t_ = remove_duplicate(t_)
                    if len(t_) and t_[0] != ' ':
                        t_ = ' ' + t_
                        
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|en|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [21]:
results = loop((files[:10], 0))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 10/10 [00:14<00:00,  1.49s/it]


In [22]:
len(results)

289

In [23]:
results[0]

{'new_text': "<|startoftranscript|><|en|><|transcribe|><|0.00|> That's what always<|3.00|><|3.00|> That sometimes<|4.00|><|4.00|> The wrong thing we always<|6.82|><|6.82|> There is always<|8.92|><|8.92|> That's why I said just about to return<|11.14|><|11.14|> In that sense I'm grateful<|13.00|><|13.00|> The term is when I<|14.92|><|14.92|> Have an example<|16.98|><|16.98|> Like Instagram or YouTube or something<|18.68|><|18.68|> It's more of me, I just want<|20.78|><|20.78|> It's like sharing<|23.40|><|23.40|> Because if I want to focus<|25.20|><|25.20|> Just for conflicts and so on<|26.96|><|26.96|> The problem is my problem in the world of health<|29.24|><|endoftext|>",
 'audio_filename': 'split-indonesian/0-1-44160000.mp3'}

In [24]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')
ori_tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
tokenizer.tokenize(results[-1]['new_text'])

['<|startoftranscript|>',
 '<|en|>',
 '<|transcribe|>',
 '<|0.00|>',
 'ĠIt',
 "'s",
 'Ġdifficult',
 ',',
 'Ġthe',
 'Ġproposal',
 'Ġis',
 'Ġover',
 '<|1.96|>',
 '<|1.96|>',
 'Ġhow',
 'Ġmany',
 '<|3.20|>',
 '<|3.20|>',
 'Ġthere',
 'Ġare',
 'Ġ23',
 ',',
 'ĠI',
 'Ġsaid',
 '<|5.84|>',
 '<|5.84|>',
 'ĠPe',
 'bi',
 'Ġwants',
 'Ġto',
 'Ġbring',
 'Ġ25',
 ',',
 'Ġcan',
 'ĠI',
 '<|7.98|>',
 '<|7.98|>',
 'Ġokay',
 'ĠI',
 "'ll",
 'Ġlook',
 'Ġfor',
 'Ġit',
 'Ġlater',
 ',',
 'ĠI',
 "'ll",
 'Ġadd',
 'Ġit',
 '<|10.08|>',
 '<|10.08|>',
 'ĠAl',
 'ham',
 'dul',
 'illah',
 ',',
 'Ġso',
 'Ġshe',
 'Ġgot',
 'Ġmarried',
 '<|11.90|>',
 '<|11.90|>',
 'Ġdoesn',
 "'t",
 'Ġbother',
 'Ġme',
 ',',
 'Ġmy',
 '<|14.36|>',
 '<|14.36|>',
 'Ġinspirational',
 'Ġwoman',
 ',',
 'Ġif',
 'Ġasked',
 '<|16.04|>',
 '<|16.04|>',
 'Ġproblem',
 ',',
 'ĠI',
 'Ġwould',
 'Ġdefinitely',
 'Ġcry',
 ',',
 'ĠI',
 'Ġremember',
 '<|17.82|>',
 '<|17.82|>',
 'ĠOh',
 'ĠAllah',
 ',',
 'Ġso',
 'Ġwhere',
 'Ġdoes',
 '<|20.06|>',
 '<|20.06|>',
 'Ġshe'

In [26]:
results = mp.multiprocessing(files, loop, cores = 50)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [27]:
len(results)

301763

In [28]:
results[0]

{'new_text': "<|startoftranscript|><|en|><|transcribe|><|0.00|> That's what always<|3.00|><|3.00|> That sometimes<|4.00|><|4.00|> The wrong thing we always<|6.82|><|6.82|> There is always<|8.92|><|8.92|> That's why I said just about to return<|11.14|><|11.14|> In that sense I'm grateful<|13.00|><|13.00|> The term is when I<|14.92|><|14.92|> Have an example<|16.98|><|16.98|> Like Instagram or YouTube or something<|18.68|><|18.68|> It's more of me, I just want<|20.78|><|20.78|> It's like sharing<|23.40|><|23.40|> Because if I want to focus<|25.20|><|25.20|> Just for conflicts and so on<|26.96|><|26.96|> The problem is my problem in the world of health<|29.24|><|endoftext|>",
 'audio_filename': 'split-indonesian/0-1-44160000.mp3'}

In [29]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [30]:
with open('prepared-indonesian-en.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 301763/301763 [00:01<00:00, 240039.63it/s]


In [31]:
!ls -lh prepared-indonesian-en.jsonl

-rw-r--r-- 1 ubuntu ubuntu 173M Apr 25 15:27 prepared-indonesian-en.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
from huggingface_hub import HfApi
api = HfApi()

In [33]:
api.upload_file(
    path_or_fileobj='prepared-indonesian-en.jsonl',
    path_in_repo='prepared-indonesian-en.jsonl',
    repo_id='mesolitica/pseudolabel-indonesian-large-v3-timestamp',
    repo_type='dataset',
)

'https://huggingface.co/datasets/mesolitica/pseudolabel-indonesian-large-v3-timestamp/blob/main/prepared-indonesian-en.jsonl'

In [34]:
results[:10]

[{'new_text': "<|startoftranscript|><|en|><|transcribe|><|0.00|> That's what always<|3.00|><|3.00|> That sometimes<|4.00|><|4.00|> The wrong thing we always<|6.82|><|6.82|> There is always<|8.92|><|8.92|> That's why I said just about to return<|11.14|><|11.14|> In that sense I'm grateful<|13.00|><|13.00|> The term is when I<|14.92|><|14.92|> Have an example<|16.98|><|16.98|> Like Instagram or YouTube or something<|18.68|><|18.68|> It's more of me, I just want<|20.78|><|20.78|> It's like sharing<|23.40|><|23.40|> Because if I want to focus<|25.20|><|25.20|> Just for conflicts and so on<|26.96|><|26.96|> The problem is my problem in the world of health<|29.24|><|endoftext|>",
  'audio_filename': 'split-indonesian/0-1-44160000.mp3'},
 {'new_text': '<|startoftranscript|><|en|><|transcribe|><|0.00|> In the world of health, no matter what we are right<|2.18|><|2.18|> He is wrong, when we show him<|4.28|><|4.28|> our truth, and we<|5.94|><|5.94|> As if we are swollen others<|7.34|><|7.34|> Th